### Define cient

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd

from minio import Minio

In [ ]:
# Define client with service account
client = Minio('s3.stxnext.pl',
                     access_key='ACCESS_KEY',
                     secret_key='SECRET_KEY',
                     secure=True)

In [ ]:
tags = client.get_bucket_tags("ml-demo")
tags

In [ ]:
type(client)

### Bucket operations

In [ ]:
# Check if bucket exists
if client.bucket_exists("ml-demo"):
    print("my-bucket exists")
else:
    print("my-bucket does not exist")

In [ ]:
# List buckets
client.list_buckets()

In [ ]:
# List objects information.
objects = client.list_objects("ml-demo")
for obj in objects:
    print(obj)

In [ ]:
# List objects information whose names starts with "my/prefix/".
objects = client.list_objects("ml-demo", prefix="data/test/")
for obj in objects:
    print(obj.object_name)
    print(obj.is_dir)
    print(obj.size)
    print(obj.metadata)

### Object operations

In [ ]:
# Get data of an object.
try:
    response = client.get_object("ml-demo", "data/test/mlmldata.csv")
    # Read data from response.
    df = pd.read_csv(response)
finally:
    response.close()
    response.release_conn()
    
    
df.head(3)

In [ ]:
# Download data of an object.
client.fget_object("ml-demo", "data/test/mlmldata.csv", "mlmldata.csv")

In [ ]:
# PUT

# Progress() -> https://github.com/minio/minio-py/blob/master/examples/progress.py

import io
from urllib.request import urlopen

# Upload data.
result = client.put_object(
    "ml-demo", "my-hello-object", io.BytesIO(b"hello"), 5,
    # progress=Progress(),
)
print(
    "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
    ),
)

# Upload unknown sized data.
data = urlopen(
    "https://cdn.kernel.org/pub/linux/kernel/v5.x/linux-5.4.81.tar.xz",
)
result = client.put_object(
    "ml-demo", "my-big-hello-object", data, length=-1, part_size=10*1024*1024,
    # progress=Progress(),
)
print(
    "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
    ),
)


In [ ]:
# FPUT

# Progress() -> https://github.com/minio/minio-py/blob/master/examples/progress.py

# Upload data with content-type.
result = client.fput_object(
    "ml-demo", "test-2/mlmldata.csv", "mlmldata.csv",
    content_type="application/csv",
    # progress=Progress(),
)
print(
    "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
    ),
)

In [ ]:
# Remove object.
client.remove_object("my-bucket", "my-object")

# Remove version of an object.
client.remove_object(
    "my-bucket", "my-object",
    version_id="dfbd25b3-abec-4184-a4e8-5a35a5c1174d",
)

In [ ]:
# Remove list of objects.
errors = client.remove_objects(
    "my-bucket",
    [
        DeleteObject("my-object1"),
        DeleteObject("my-object2"),
        DeleteObject("my-object3", "13f88b18-8dcd-4c83-88f2-8631fdb6250c"),
    ],
)
for error in errors:
    print("error occured when deleting object", error)

# Remove a prefix recursively.
delete_object_list = map(
    lambda x: DeleteObject(x.object_name),
    client.list_objects("my-bucket", "my/prefix/", recursive=True),
)
errors = client.remove_objects("my-bucket", delete_object_list)
for error in errors:
    print("error occured when deleting object", error)

In [ ]:
from datetime import datetime, timezone
from minio.commonconfig import REPLACE, CopySource

# # copy an object from a bucket to another.
# result = client.copy_object(
#     "my-bucket",
#     "my-object",
#     CopySource("my-sourcebucket", "my-sourceobject"),
# )
# print(result.object_name, result.version_id)

# # copy an object with condition.
# result = client.copy_object(
#     "my-bucket",
#     "my-object",
#     CopySource(
#         "my-sourcebucket",
#         "my-sourceobject",
#         modified_since=datetime(2014, 4, 1, tzinfo=timezone.utc),
#     ),
# )
# print(result.object_name, result.version_id)

In [ ]:
# Select content of an object by SQL expression.
# with client.select_object_content(
#         "ml-demo",
#         "data/test/mlmldata.csv",
#         SelectRequest(
#             "select * from S3Object",
#             CSVInputSerialization(),
#             CSVOutputSerialization(),
#             request_progress=True,
#         ),
# ) as result:
#     for data in result.stream():
#         print(data.decode())
#     print(result.stats())